### Deploy Trained Model - Azure Machine Learning Service

In [7]:
import azureml
from azureml.core import Workspace, Run
from azureml.core.model import Model

In [8]:
ws = Workspace.from_config()

Found the config file in: /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/config.json


In [9]:
output_folder = "outputs"
model_name = "vehicle_recommendations_model"
item_mapping_model_name = 'item_to_veh_mappings'
veh_mapping_model_name = 'veh_to_item_mappings'

### Create Environment File

In [10]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("torch")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())


### Create Congifuration File

In [11]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Vehicle_Recommendations",  "method" : "GNN"}, 
                                               description='Session based vehicle recommendations using GNN')

In [13]:
model1=Model(ws, model_name)
model2=Model(ws, item_mapping_model_name)
model3=Model(ws, veh_mapping_model_name)

### Create the Image

In [15]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                  dependencies=["model.py"])

# Register the image from the image configuration
image = ContainerImage.create(name = "veh-recommendations-init",
                              models = [model1, model2, model3], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )

Creating image
CPU times: user 320 ms, sys: 33.6 ms, total: 354 ms
Wall time: 5.79 s


### Update the Image in Webservice

In [18]:
from azureml.core.webservice import Webservice
from azureml.core.image import Image

service = Webservice(ws, "veh-recommendations-init")
image = Image(ws, name="veh-recommendations-init")

service.update(image=image)